In [ ]:
import pandas as pd
import pickle
import os
import matplotlib.pyplot as plt
import seaborn as sns

from dieboldmariano import dm_test

In [ ]:
# --- CONFIGURAÇÕES GLOBAIS ---

MODEL_NAMES = [
    'HyS-MF',
    'HyS-MF_MIMO',
    'ARIMA-MLP',
    'ARIMA-LSTM',
    'PureARIMA',
    'PureLSTM',
    'PureNBEATS'
]
DATASET_NAMES = [
    'AirPassengers', 'co2', 'UKgas', 'Sunspots','ukdriverdeaths'
]
FORECAST_HORIZON = 10

# Caminhos para os resultados
METRICS_PATH = 'results/metrics/'
PREDICTIONS_PATH = 'results/predictions/'

# Estilo visual para os gráficos
sns.set_style("whitegrid")
print("Ambiente de análise configurado com sucesso!")

In [ ]:
all_metrics = []
for ds_name in DATASET_NAMES:
    row = {'Dataset': ds_name}
    for model_name in MODEL_NAMES:
        file_path = os.path.join(METRICS_PATH, f"{ds_name}_{model_name}.pkl")
        if os.path.exists(file_path):
            with open(file_path, 'rb') as f:
                metrics = pickle.load(f)
                row[model_name] = metrics
    all_metrics.append(row)

# Criar um DataFrame base com as métricas
df_metrics = pd.DataFrame(all_metrics).set_index('Dataset')

# --- Tabela de MAPE ---
df_mape = df_metrics.map(lambda x: x.get('MAPE') if isinstance(x, dict) else None)
print("Tabela de Resultados: Mean Absolute Percentage Error (MAPE)")
display(df_mape.style.highlight_min(axis=1, color='blue').format("{:.2f}"))

# --- Tabela de MASE ---
df_mase = df_metrics.map(lambda x: x.get('MASE') if isinstance(x, dict) else None)
print("\nTabela de Resultados: Mean Absolute Scaled Error (MASE)")
display(df_mase.style.highlight_min(axis=1, color='blue').format("{:.3f}"))

In [ ]:
DS_TO_PLOT = 'AirPassengers'

try:
    full_series_df = pd.read_csv(f"datasets/{DS_TO_PLOT}.csv", index_col=0, parse_dates=True)
    test_series = full_series_df[-FORECAST_HORIZON:].squeeze().rename('Valores Reais')

    # Plotar os dados reais de teste
    plt.figure(figsize=(14, 7))
    plt.plot(test_series.index, test_series.values, label='Valores Reais', color='black', linewidth=2.5, zorder=10)

    # Plotar as previsões de cada modelo
    for model_name in MODEL_NAMES:
        file_path = os.path.join(PREDICTIONS_PATH, f"{DS_TO_PLOT}_{model_name}.csv")
        if os.path.exists(file_path):
            pred_df = pd.read_csv(file_path, index_col=0, parse_dates=True)
            plt.plot(pred_df.index, pred_df['prediction'], label=f'Previsão {model_name}', linestyle='--')

    plt.title(f'Comparação de Previsões para o Dataset {DS_TO_PLOT}', fontsize=16)
    plt.xlabel('Data', fontsize=12)
    plt.ylabel('Valores', fontsize=12)
    plt.legend()
    plt.show()

except FileNotFoundError:
    print(f"Dataset '{DS_TO_PLOT}.csv' não encontrado na pasta 'datasets/'. Verifique o nome ou execute o notebook de treinamento.")

In [ ]:
dm_results_summary = []

for ds_name in DATASET_NAMES:
    # Dicionário para guardar as previsões de cada modelo
    predictions_dict = {}
    try:
        # Carregar os valores reais do dataset
        full_series_df = pd.read_csv(f"datasets/{ds_name}.csv", index_col=0, parse_dates=True)
        real_values = full_series_df[-FORECAST_HORIZON:].squeeze().values

        # Carregar as previsões de todos os modelos
        for model_name in MODEL_NAMES:
            file_path = os.path.join(PREDICTIONS_PATH, f"{ds_name}_{model_name}.csv")
            if os.path.exists(file_path):
                predictions_dict[model_name] = pd.read_csv(file_path, index_col=0, parse_dates=True)['prediction'].values

        # Comparar o HyS-MF com todos os outros modelos para este dataset
        base_model_preds = predictions_dict.get('HyS-MF')
        
        if base_model_preds is not None:
            row = {'Dataset': ds_name}
            for model_name, model_preds in predictions_dict.items():
                if model_name != 'HyS-MF':
                    # --- CORREÇÃO APLICADA AQUI ---
                    # Passando os valores reais e as duas previsões para o teste
                    dm_stat, p_value = dm_test(
                        real_values, 
                        base_model_preds, 
                        model_preds, 
                        one_sided="less" # Testa se o erro do HyS-MF (pred1) é menor que o do outro modelo (pred2)
                    )
                    row[f'{model_name}_p-valor'] = p_value
            dm_results_summary.append(row)
            
    except FileNotFoundError:
        print(f"Pulando teste DM para o dataset '{ds_name}' (arquivo não encontrado).")
    except KeyError:
        print(f"Pulando teste DM para o dataset '{ds_name}' (previsão do HyS-MF não encontrada).")


df_dm = pd.DataFrame(dm_results_summary).set_index('Dataset')

# Função para colorir p-valores significativos
def color_significant(val):
    # Verifica se o valor é um número antes de comparar
    if isinstance(val, (int, float)) and val < 0.05:
        return 'background-color: blue'
    return ''

print("\nTabela de p-valores do Teste Diebold-Mariano (H1: HyS-MF é melhor)")
print("Valores < 0.05 (azul) indicam que a melhoria do HyS-MF é estatisticamente significativa.")
display(df_dm.style.map(color_significant).format("{:.4f}"))

In [ ]:
# Célula 5: Gráfico de Ganho Percentual (Similar à Figura 9)

# O DataFrame deve conter apenas os valores numéricos de MAPE, sem o '%' no final.

# Vamos recriar o df_mape numericamente para garantir
df_mape_numeric = df_metrics.map(lambda x: x.get('MAPE') if isinstance(x, dict) else None)

# 1. Calcular a média do MAPE para cada modelo em todos os datasets
mean_mapes = df_mape_numeric.mean()

# 2. Identificar o melhor modelo (aquele com o menor MAPE médio)
best_model_name = mean_mapes.idxmin()
best_model_mape = mean_mapes.min()

print(f"O melhor modelo em média foi: '{best_model_name}' com um MAPE médio de {best_model_mape:.2f}%")

# 3. Calcular a Diferença Percentual (PD) para todos os outros modelos
# Fórmula: PD = 100 * (MAPE_outro_modelo - MAPE_melhor_modelo) / MAPE_outro_modelo
# Esta fórmula é uma variação da do artigo para mostrar o ganho.
pd_results = {}
for model_name, mape_a in mean_mapes.items():
    if model_name != best_model_name:
        # Onde 'a' é o modelo de comparação e 'b' é o melhor modelo
        pd_value = 100 * (mape_a - best_model_mape) / mape_a
        pd_results[model_name] = pd_value

# 4. Preparar os dados para o gráfico
df_pd = pd.DataFrame.from_dict(pd_results, orient='index', columns=['PD(%)'])
df_pd_sorted = df_pd.sort_values(by='PD(%)', ascending=True)

# 5. Gerar o gráfico de barras
plt.figure(figsize=(12, 8))
barplot = sns.barplot(
    x=df_pd_sorted.index, 
    y=df_pd_sorted['PD(%)'], 
    palette='viridis',
    hue=df_pd_sorted.index,  # Atribui a variável de 'x' para 'hue'
    legend=False             # Impede a criação da legenda redundante
)

plt.title(f'Diferença Percentual (PD) do MAPE em Relação ao Melhor Modelo ({best_model_name})', fontsize=16)
plt.ylabel('PD (%)', fontsize=12)
plt.xlabel('Modelo de Comparação', fontsize=12)
plt.xticks(rotation=45, ha='right') # Rotaciona os nomes dos modelos para não sobrepor
plt.tight_layout() # Ajusta o layout para caber tudo

# Adicionar os valores no topo das barras
for p in barplot.patches:
    barplot.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')

plt.show()